In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.5'
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

zsh:1: command not found: apt-get
zsh:1: command not found: apt-get
zsh:1: command not found: wget
tar: Error opening archive: Failed to open 'spark-3.4.3-bin-hadoop3.tgz'


Exception: Unable to find py4j in /content/spark-3.4.3-bin-hadoop3/python, your SPARK_HOME may not be configured correctly

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_price_4_bedrooms = spark.sql("""
    SELECT year(date) as year_sold, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year_sold
    ORDER BY year_sold
""")
avg_price_4_bedrooms.show()


+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_3_beds_3_baths = spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
""")
avg_price_3_beds_3_baths.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_large_home = spark.sql("""
    SELECT date_built, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""")
avg_price_large_home.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

avg_price_per_view = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""")
avg_price_per_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 1.1306045055389404 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

avg_price_per_view_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""")
avg_price_per_view_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 2.1939380168914795 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
output_parquet_path = "/mnt/data/home_sales_partitioned_by_date_built"
df.write.partitionBy("date_built").parquet(output_parquet_path)

In [ ]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet(output_parquet_path)

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

avg_price_per_view_parquet = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) as avg_price
    FROM home_sales_parquet
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
""")
avg_price_per_view_parquet.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  91|1137372.73|
|  97|1129040.15|
|  84|1117233.13|
|  75|1114042.94|
|  89|1107839.15|
|  78|1080649.37|
|  77|1076205.56|
|  87| 1072285.2|
|  86|1070444.25|
|  82| 1063498.0|
|  90|1062654.16|
|  99|1061201.42|
|  76|1058802.78|
|  85|1056336.74|
|  95| 1054325.6|
|  98|1053739.33|
|  81|1053472.79|
|  83|1033965.93|
|  94| 1033536.2|
|  88|1031719.35|
+----+----------+
only showing top 20 rows

--- 1.2142176628112793 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [ ]:
# 15. Check if the home_sales is no longer cached
print("Is 'home_sales' cached after uncache?:", spark.catalog.isCached("home_sales"))

Is 'home_sales' cached after uncache?: False
